# PDG ATTACK 

### *Import*

In [1]:
import numpy as np
import torch
from torch.nn import CrossEntropyLoss
from PIL import Image
from torchvision import transforms
from art.attacks.evasion import ProjectedGradientDescentPyTorch
from art.estimators.classification import PyTorchClassifier
import matplotlib.pyplot as plt
import tensorflow as tf

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

c:\Users\cassd\miniconda3\envs\ai4cyber\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


### initialization


In [2]:
if str(device) in 'cuda':
    print("Import Inception.Inception")
    import inception
    resnet = inception.InceptionResnetV1(pretrained='vggface2').eval()
elif str(device) == "cpu":
    print("Import Facenet.Inception")
    from facenet_pytorch import InceptionResnetV1
    resnet = InceptionResnetV1(pretrained='vggface2').eval()

Import Inception.Inception


In [3]:
from utils import get_labels


resnet.classify = True
resnet.to(device)
LABELS = get_labels()

In [4]:

from torch.nn import CrossEntropyLoss
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import ProjectedGradientDescentPyTorch


#Impostare l'input shape
model = PyTorchClassifier(resnet,input_shape=[224,224], loss=CrossEntropyLoss(),nb_classes=8631, device_type=device) #This class implements a classifier with the PyTorch framework.


### Untargeted Attack on all test set

In [ ]:
# PGD generic Attack for all samples
import os
import re
from utils import load_image_NN1

dataset_dir = "test_set_cropped_piccolo/" 
eps_step = [2e-05, 5e-05, 7e-05, 2e-03, 5e-03, 7e-03, 2e-02, 5e-02, 7e-02, 1e-01] #La dimensione del passo della variazione dell'input a ogni iterazione. Questo parametro definisce quanto grande sarà ogni passo nella direzione del gradiente. #La dimensione del passo della variazione dell'input per il calcolo della perturbazione minima. Questo parametro è utilizzato quando minimal è impostato su True.
max_iter = [2,5,7]  # Il numero massimo di iterazioni. Questo parametro determina quante volte l'attacco iterativo sarà eseguito.
 
eps_range = 0.1 #Questo valore determina l'ampiezza massima della perturbazione aggiunta agli input originali. Può essere un valore singolo (int o float) o un array numpy (ndarray).

accuracy_for_eps = np.zeros((len(max_iter),len(eps_step)))

perturbation_for_eps = np.zeros((len(max_iter),len(eps_step)))

correct_predictions = 0
total_images = 0
print("Inizio Attacco PGD NON-TARGETED")
for i in range(len(max_iter)):
        print("**** Attacco con max_iter {} **** ".format(max_iter[i]))
        for j in range(len(eps_step)):   #Se qualcosa funziona strano controllare questo zip
            correct_predictions = 0
            total_images = 0
            perturbation = []
            
            attack = ProjectedGradientDescentPyTorch(estimator=model, eps = eps_range, eps_step=eps_step[j], targeted=False, max_iter = max_iter[i])
            
            print("**** Attacco con eps:{} con step a {} ****".format(eps_range,eps_step[j]))
            for filename in os.listdir(dataset_dir):
                if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                    person_path = os.path.join(dataset_dir, filename)
                    test_img = load_image_NN1(person_path)
                    test_img = test_img.numpy()
                    test_images_adv = attack.generate(test_img)
                    model_predictions = model.predict(test_images_adv)
                    correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
                    print("Etichetta corretta:", correct_label)   
                    perturbation.append(np.mean(np.abs((test_images_adv - test_img))))  #Salvo le perturbazioni applicate su ogni immagine
                    predicted_label = LABELS[np.array(model_predictions[0].argmax())]
                    print("Predetto {} con probabilità {} e con perturbazione {}".format(predicted_label,model_predictions[0][model_predictions.argmax()],perturbation[-1]))
                    total_images+=1
                    
                    predicted_label = str(predicted_label)

                    if correct_label in predicted_label:
                        correct_predictions+=1

                    accuracy = correct_predictions/total_images
                    print("Adversarial Sample misclassificati correttamente attuale: {}%".format((100-(accuracy*100))))
            

            if total_images != 0:    #Per ogni epsilon impostata salvo in accuracy plot
                if len(perturbation) == total_images:
                    perturbazione_media = sum(perturbation)/total_images    #Calcolo la media delle perturbazioni applicate su tutte le immagini per una determinata epsilon
                    perturbation_for_eps[i][j] = perturbazione_media
                    print("----------- Perturbazione media aggiunta a tutte le immagini per eps_Step: {} equivale a {}% ----------------".format(eps_step[j],perturbazione_media))

                final_accuracy = correct_predictions/total_images
                accuracy_for_eps[i][j] = final_accuracy
                print("----------- Accuracy sugli adversarial Sample per eps_step: {} equivale a {}% ----------------".format(eps_step[j],final_accuracy))
                print("")
            #Per ogni epsilon impostata salvo in accuracy plot
        print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------")
        print()
# correct_predictions = 0
# total_images = 0
# for filename in os.listdir(dataset_dir):
#     if filename.endswith(".jpg") or filename.endswith(".jpeg"):
#         person_path = os.path.join(dataset_dir, filename)
#         print("Immagine:", filename)
#         test_img = load_image(person_path)
#         test_img = test_img.unsqueeze(0)
#         test_img = test_img.numpy()
#         test_images_adv = attack.generate(test_img)
#         model_predictions = model.predict(test_images_adv)
#         correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
#         print("Etichetta corretta:", correct_label)   
#         perturbation = np.mean(np.abs((test_images_adv - test_img)))
#         predicted_label = LABELS[np.array(model_predictions[0].argmax())]
#         print("Predetto {} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))
#         total_images+=1

#         if predicted_label == correct_label:
#             correct_predictions+=1

#         accuracy = correct_predictions/total_images
#         print("Accuracy sugli adversarial Sample: {}%".format((100-(accuracy*100))))
        

# if total_images != 0:
#     final_accuracy = correct_predictions/total_images
#     print("----------- Accuracy FINALE sugli adversarial Sample: {}\% ----------------".format(final_accuracy))

In [ ]:
from matplotlib import pyplot as plt

# plot accuracy/Attack strength

fig, ax = plt.subplots()
ax.plot(np.array(eps_range), np.array(accuracy_for_eps), 'b--', label='NN1')

legend = ax.legend(loc='upper center', shadow=True, fontsize='large')
legend.get_frame().set_facecolor('#00FFCC')

plt.xlabel('Attack strength (eps)')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.array(eps_range), np.array(perturbation_for_eps), 'b--', label='NN1')

legend = ax.legend(loc='upper center', shadow=True, fontsize='large')
legend.get_frame().set_facecolor('#00FFCC')

plt.xlabel('Attack strength (eps)')
plt.ylabel('mean perturbation')
plt.show()

### Single sample Untargeted attack

In [ ]:
epsilon = 1
eps_step = 0.005
max_iter = 5 

attack = ProjectedGradientDescentPyTorch(estimator=model, eps = epsilon, eps_step= eps_step, targeted=True, max_iter = max_iter)

In [ ]:
from utils import load_image_NN1
import os
import numpy as np

target_class = 10

dataset_dir = "test_set_cropped/"
filename = "Brad_Pitt_8_face_0.jpg"
person_path = os.path.join(dataset_dir, filename)

test_img = load_image_NN1(person_path)

target_class = 10
etichetta_target = LABELS[0]
print("ETICHETTA TARGET: ", LABELS[10])

batch_size = test_img.shape[0]
print(test_img.shape[0])
targeted_labels = np.array([target_class] * batch_size)
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes=8631)



test_img_numpy = test_img.numpy()
test_images_adv = attack.generate(test_img_numpy, one_hot_targeted_labels)
model_predictions = model.predict(test_images_adv)



model_predictions = model.predict(test_images_adv)
perturbation = np.mean(np.abs((test_images_adv - test_img_numpy)))
print('Average perturbation: {:4.2f}'.format(perturbation))
#targeted_attack_loss, targeted_attack_accuracy = model.evaluate(test_images_adv, targeted_labels)
#print('Targeted attack accuracy: {:4.2f}'.format(targeted_attack_accuracy))
print("Etichetta target:{}".format(LABELS[target_class]))
predicted_label = LABELS[np.array(model_predictions.argmax())]
print("{} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))

In [ ]:
# Prepara le immagini per la visualizzazione
# Rimuovi la dimensione batch extra e converti nel formato channels-last
test_images_adv = np.squeeze(test_images_adv, axis=0)
test_images_adv = np.transpose(test_images_adv, (1, 2, 0))

# Converti le immagini in uint8 per la visualizzazione
if test_img_numpy.dtype != np.uint8:
    test_img_numpy = (test_img_numpy * 255).astype(np.uint8)
    test_img_numpy = np.squeeze(test_img_numpy, axis=0)  # Rimuovi la dimensione batch extra
    test_img_numpy = np.transpose(test_img_numpy, (1, 2, 0))

if test_images_adv.dtype != np.uint8:
    test_images_adv = (test_images_adv * 255).astype(np.uint8)

# Visualizza le immagini affiancate con Matplotlib
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Mostra l'immagine originale
ax1.imshow(test_img_numpy)
ax1.set_title('Original Image')
ax1.axis('off')

# Mostra l'immagine avversaria
ax2.imshow(test_images_adv)
ax2.set_title(f'Adversarial Image\nPredicted: {predicted_label}')
ax2.axis('off')

# Mostra la figura
plt.suptitle("DeepFool Adversarial Images")
plt.show()


### Targeted Attack on all test set

In [6]:
#PGD specific Attack for all samples
import os
import re
import torch
from utils import load_image_NN1

dataset_dir = "test_set_cropped_piccolo/" 
eps_step = [2e-05, 5e-05, 7e-05, 2e-03, 5e-03, 7e-03, 2e-02, 5e-02, 7e-02, 1e-01] #La dimensione del passo della variazione dell'input a ogni iterazione. Questo parametro definisce quanto grande sarà ogni passo nella direzione del gradiente. #La dimensione del passo della variazione dell'input per il calcolo della perturbazione minima. Questo parametro è utilizzato quando minimal è impostato su True.

max_iter = [2,5,7]  # Il numero massimo di iterazioni. Questo parametro determina quante volte l'attacco iterativo sarà eseguito.

eps_range = 0.1  # Questo valore determina l'ampiezza massima della perturbazione aggiunta agli input originali. Può essere un valore singolo (int o float) o un array numpy (ndarray).

accuracy_misclassified_for_eps = np.zeros((len(max_iter),len(eps_step)))

accuracy_for_eps = np.zeros((len(max_iter),len(eps_step)))

perturbation_for_eps = np.zeros((len(max_iter),len(eps_step)))

correct_misclassified = 0
total_images = 0

target_class = 10
etichetta_target = LABELS[0]
print("ETICHETTA TARGET: ", LABELS[10])
shape = 1 #shape di test image 
batch_size = shape
targeted_labels = np.array([target_class] * batch_size)
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes=8631)

print("Inizio Attacco PGD NON-TARGETED")
for i in range(len(max_iter)):
        print("**** Attacco con max_iter {} **** ".format(max_iter[i]))
        for j in range(len(eps_step)):   #Se qualcosa funziona strano controllare questo zip
            correct_predictions = 0
            total_images = 0
            perturbation = []
            
            attack = ProjectedGradientDescentPyTorch(estimator=model, eps = eps_range, eps_step=eps_step[j], targeted=True, max_iter = max_iter[i])
            
            print("**** Attacco con eps:{} con step a {} ****".format(eps_range,eps_step[j]))
            for filename in os.listdir(dataset_dir):
                if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                    person_path = os.path.join(dataset_dir, filename)
                    test_img = load_image_NN1(person_path)
                    test_img = test_img.numpy()
                    test_images_adv = attack.generate(test_img, one_hot_targeted_labels)
                    model_predictions = model.predict(test_images_adv)
                    correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
                    print("Etichetta corretta:", correct_label)   
                    perturbation.append(np.mean(np.abs((test_images_adv - test_img))))  #Salvo le perturbazioni applicate su ogni immagine
                    predicted_label = LABELS[np.array(model_predictions[0].argmax())]
                    print("Predetto {} con probabilità {} e con perturbazione {}".format(predicted_label,model_predictions[0][model_predictions.argmax()],perturbation[-1]))
                    total_images+=1
                    
                    predicted_label = str(predicted_label)

                    if correct_label in predicted_label:
                        correct_predictions+=1
                        
                    if etichetta_target in predicted_label:  
                        correct_misclassified = correct_misclassified+1   #Se il modello predice l'etichetta target allora è correttamente misclassificato

                    accuracy_misclassified = correct_misclassified/total_images
                    print("Adversarial Sample misclassificati correttamente attualmente: {}%".format((accuracy_misclassified)))
                    print("Accuracy attuale: {}%".format((correct_predictions/total_images)*100))
                    print("")

            if total_images != 0:    #Per ogni epsilon impostata salvo in accuracy plot
                if len(perturbation) == total_images:
                    perturbazione_media = sum(perturbation)/total_images    #Calcolo la media delle perturbazioni applicate su tutte le immagini per una determinata epsilon
                    perturbation_for_eps[i][j] = perturbazione_media
                    print("----------- Perturbazione media aggiunta a tutte le immagini per eps_Step: {} equivale a {}% ----------------".format(eps_step[j],perturbazione_media))

                final_accuracy = correct_predictions/total_images
                accuracy_for_eps[i][j] = final_accuracy
                print("----------- Accuracy sugli adversarial Sample per eps_step: {} equivale a {}% ----------------".format(eps_step[j],final_accuracy))
                
                accuracy_misclassified = correct_misclassified/total_images
                accuracy_misclassified_for_eps[i][j] = accuracy_misclassified
                print("----------- Adversarial Sample misclassificati correttamente: {}% -----------".format((accuracy_misclassified)))
                print("")
            #Per ogni epsilon impostata salvo in accuracy plot
        print("")

        print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------")
        print("")

# correct_predictions = 0
# total_images = 0
# target_class = 10
# print("ETICHETTA TARGET: ", LABELS[10])
# batch_size = test_img.shape[0]
# targeted_labels = np.array([target_class] * batch_size)
# one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes=8631)

# for filename in os.listdir(dataset_dir):
#     if filename.endswith(".jpg") or filename.endswith(".jpeg"):
#         person_path = os.path.join(dataset_dir, filename)
#         print("Immagine:", filename)
#         test_img = load_image(person_path)
#         test_img = test_img.unsqueeze(0)
#         test_img = test_img.numpy()
#         test_images_adv = attack.generate(test_img, one_hot_targeted_labels)
#         model_predictions = model.predict(test_images_adv)
#         correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
#         print("Etichetta corretta:", correct_label)   
#         perturbation = np.mean(np.abs((test_images_adv - test_img)))
#         predicted_label = LABELS[np.array(model_predictions[0].argmax())]
#         print("Predetto {} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))
#         total_images+=1

#         if predicted_label == correct_label:
#             correct_predictions+=1

#         accuracy = correct_predictions/total_images
#         print("Accuracy sugli adversarial Sample: {}%".format((100-(accuracy*100))))
        

# if total_images != 0:
#     final_accuracy = correct_predictions/total_images
#     print("----------- Accuracy FINALE sugli adversarial Sample: {}\% ----------------".format(final_accuracy))

ETICHETTA TARGET:   Aaron_Hernandez
Inizio Attacco PGD NON-TARGETED
**** Attacco con max_iter 2 **** 
**** Attacco con eps:0.1 con step a 2e-05 ****


Etichetta corretta: Andrea_Pirlo
Predetto  Andrea_Pirlo con probabilità 12.70524787902832 e con perturbazione 3.815378659055568e-05
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 100.0%



Etichetta corretta: Antonio_Cassano
Predetto  Branch_Warren con probabilità 10.500913619995117 e con perturbazione 3.808758992818184e-05
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 50.0%



Etichetta corretta: Ariana_Grande
Predetto  Ariana_Grande con probabilità 11.141061782836914 e con perturbazione 3.7927784433122724e-05
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 66.66666666666666%



Etichetta corretta: Ashley_Scott
Predetto  Ashley_Scott con probabilità 13.847076416015625 e con perturbazione 3.818137338384986e-05
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 75.0%



...

Etichetta corretta: Ashley_Scott
Predetto  Aaron_Hernandez con probabilità 12.574553489685059 e con perturbazione 0.06109701097011566
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 0.0%



Etichetta corretta: Boris_Johnson
Predetto  Aaron_Hernandez con probabilità 15.265563011169434 e con perturbazione 0.060866545885801315
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 0.0%



Etichetta corretta: Brad_Pitt
Predetto  Aaron_Hernandez con probabilità 15.537111282348633 e con perturbazione 0.060865242034196854
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 0.0%



Etichetta corretta: Christian_De_Sica
Predetto  Aaron_Hernandez con probabilità 12.705669403076172 e con perturbazione 0.05979752913117409
Adversarial Sample misclassificati correttamente attualmente: 0.0%
Accuracy attuale: 0.0%



PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from matplotlib import pyplot as plt

# plot accuracy/Attack strength

fig, ax = plt.subplots()
ax.plot(np.array(eps_range), np.array(accuracy_for_eps), 'b--', label='NN1')

legend = ax.legend(loc='upper center', shadow=True, fontsize='large')
legend.get_frame().set_facecolor('#00FFCC')

plt.xlabel('Attack strength (eps)')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.array(eps_range), np.array(perturbation_for_eps), 'b--', label='NN1')

legend = ax.legend(loc='upper center', shadow=True, fontsize='large')
legend.get_frame().set_facecolor('#00FFCC')

plt.xlabel('Attack strength (eps)')
plt.ylabel('mean perturbation')
plt.show()

### Single sample Targeted attack

In [ ]:
epsilon = 1
eps_step = 0.005
max_iter = 5 

attack = ProjectedGradientDescentPyTorch(estimator=model, eps = epsilon, eps_step= eps_step, targeted=True, max_iter = max_iter)

In [ ]:
from utils import load_image_NN1
import os
import numpy as np

target_class = 10

dataset_dir = "test_set_cropped/"
filename = "Brad_Pitt_8_face_0.jpg"
person_path = os.path.join(dataset_dir, filename)

test_img = load_image_NN1(person_path)
target_class = 10
etichetta_target = LABELS[0]
print("ETICHETTA TARGET: ", LABELS[10])
batch_size = test_img.shape[0]
targeted_labels = np.array([target_class] * batch_size)
one_hot_targeted_labels = tf.keras.utils.to_categorical(targeted_labels, num_classes=8631)
print(one_hot_targeted_labels.shape)


test_img_numpy = test_img.numpy()
test_images_adv = attack.generate(test_img_numpy, one_hot_targeted_labels)
model_predictions = model.predict(test_images_adv)



model_predictions = model.predict(test_images_adv)
perturbation = np.mean(np.abs((test_images_adv - test_img_numpy)))
print('Average perturbation: {:4.2f}'.format(perturbation))
#targeted_attack_loss, targeted_attack_accuracy = model.evaluate(test_images_adv, targeted_labels)
#print('Targeted attack accuracy: {:4.2f}'.format(targeted_attack_accuracy))
print("Etichetta target:{}".format(LABELS[target_class]))
predicted_label = LABELS[np.array(model_predictions.argmax())]
print("{} con probabilità {}".format(predicted_label,model_predictions[0][model_predictions.argmax()]))

In [ ]:
# Prepara le immagini per la visualizzazione
# Rimuovi la dimensione batch extra e converti nel formato channels-last
test_images_adv = np.squeeze(test_images_adv, axis=0)
test_images_adv = np.transpose(test_images_adv, (1, 2, 0))

# Converti le immagini in uint8 per la visualizzazione
if test_img_numpy.dtype != np.uint8:
    test_img_numpy = (test_img_numpy * 255).astype(np.uint8)
    test_img_numpy = np.squeeze(test_img_numpy, axis=0)  # Rimuovi la dimensione batch extra
    test_img_numpy = np.transpose(test_img_numpy, (1, 2, 0))

if test_images_adv.dtype != np.uint8:
    test_images_adv = (test_images_adv * 255).astype(np.uint8)

# Visualizza le immagini affiancate con Matplotlib
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Mostra l'immagine originale
ax1.imshow(test_img_numpy)
ax1.set_title('Original Image')
ax1.axis('off')

# Mostra l'immagine avversaria
ax2.imshow(test_images_adv)
ax2.set_title(f'Adversarial Image\nPredicted: {predicted_label}')
ax2.axis('off')

# Mostra la figura
plt.suptitle("DeepFool Adversarial Images")
plt.show()